# Overview

We wanted to make it easy to analyze popular songs from Spotify. We found data for the top 50 tracks for the years 2020 and 2021 and did some cleaning and transformation to load them into a postgreSQL database where they can be analyzed further. This process could be repeated for future years to build on the dataset.

# Extract

Data was downloaded from Kaggle as a CSV file:
* [Spotify Top 50 Tracks for 2020](https://www.kaggle.com/datasets/atillacolak/top-50-spotify-tracks-2020)
* [Spotify Top 50 Tracks for 2021](https://www.kaggle.com/datasets/equinxx/spotify-top-50-songs-in-2021) 

CSV files were saved in the `data` directory of the project and read using pandas.

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from config import db_password
import datetime

In [2]:
file_2021 = "data/spotify_top50_2021.csv"
file_2020 = "data/spotify_toptracks_2020.csv"

# Transform

The 2020 and 2021 Spotify top 50 tracks datasets were cleaned and combined into one database.

## Cleaning

* A year column was added to both dataframes. 
* In the 2021 dataframe, the 'id' column was renamed to 'rank' and the 'artist_nam'e column was renamed to 'artist'. 
* In the 2020 dataframe, the 'Unnamed: 0' was renamed to 'rank'.
* The 2020 dataframe rank column was edited so that the first place ranking is 1.

In [3]:
df_2021 = pd.read_csv(file_2021)
df_2021['year'] = 2021
df_2021.rename(columns={'id':"rank","artist_name":"artist"}, inplace=True)
df_2021.head()

,rank,artist,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
0,1,Olivia Rodrigo,drivers license,5wANPM4fQCJwkGd4rN57mH,92,0.561,0.431,10,-8.810,1,0.0578,0.76800,0.000014,0.1060,0.137,143.875,242013,4,2021
1,2,Lil Nas X,MONTERO (Call Me By Your Name),1SC5rEoYDGUK4NfG82494W,90,0.593,0.503,8,-6.725,0,0.2200,0.29300,0.000000,0.4050,0.710,178.781,137704,4,2021
2,3,The Kid LAROI,STAY (with Justin Bieber),5PjdY0CKGZdEuoNab3yDmX,92,0.591,0.764,1,-5.484,1,0.0483,0.03830,0.000000,0.1030,0.478,169.928,141806,4,2021
3,4,Olivia Rodrigo,good 4 u,4ZtFanR9U6ndgddUvNcjcG,95,0.563,0.664,9,-5.044,1,0.1540,0.33500,0.000000,0.0849,0.688,166.928,178147,4,2021
4,5,Dua Lipa,Levitating (feat. DaBaby),5nujrmhLynf4yMoMtj8AQF,89,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.000000,0.0674,0.915,102.977,203064,4,2021


In [4]:
df_2020 = pd.read_csv(file_2020)
df_2020['year'] = 2020
df_2020.rename(columns={'Unnamed: 0':"rank"}, inplace=True)
df_2020.head()

,rank,artist,album,track_name,track_id,energy,danceability,key,loudness,acousticness,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,genre,year
0,0,The Weeknd,After Hours,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,0.730,0.514,1,-5.934,0.00146,0.0598,0.000095,0.0897,0.334,171.005,200040,R&B/Soul,2020
1,1,Tones And I,Dance Monkey,Dance Monkey,1rgnBhdG2JDFTbYkYRZAku,0.593,0.825,6,-6.401,0.68800,0.0988,0.000161,0.1700,0.540,98.078,209755,Alternative/Indie,2020
2,2,Roddy Ricch,Please Excuse Me For Being Antisocial,The Box,0nbXyq5TXYPCO7pr3N8S4I,0.586,0.896,10,-6.687,0.10400,0.0559,0.000000,0.7900,0.642,116.971,196653,Hip-Hop/Rap,2020
3,3,SAINt JHN,Roses (Imanbek Remix),Roses - Imanbek Remix,2Wo6QQD1KMDWeFkkjLqwx5,0.721,0.785,8,-5.457,0.01490,0.0506,0.004320,0.2850,0.894,121.962,176219,Dance/Electronic,2020
4,4,Dua Lipa,Future Nostalgia,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,0.793,0.793,11,-4.521,0.01230,0.0830,0.000000,0.0951,0.679,123.950,183290,Nu-disco,2020


In [5]:
# start rank at 1 instead of 0
df_2020['rank'] = df_2020['rank'] + 1
df_2020.head()

,rank,artist,album,track_name,track_id,energy,danceability,key,loudness,acousticness,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,genre,year
0,1,The Weeknd,After Hours,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,0.730,0.514,1,-5.934,0.00146,0.0598,0.000095,0.0897,0.334,171.005,200040,R&B/Soul,2020
1,2,Tones And I,Dance Monkey,Dance Monkey,1rgnBhdG2JDFTbYkYRZAku,0.593,0.825,6,-6.401,0.68800,0.0988,0.000161,0.1700,0.540,98.078,209755,Alternative/Indie,2020
2,3,Roddy Ricch,Please Excuse Me For Being Antisocial,The Box,0nbXyq5TXYPCO7pr3N8S4I,0.586,0.896,10,-6.687,0.10400,0.0559,0.000000,0.7900,0.642,116.971,196653,Hip-Hop/Rap,2020
3,4,SAINt JHN,Roses (Imanbek Remix),Roses - Imanbek Remix,2Wo6QQD1KMDWeFkkjLqwx5,0.721,0.785,8,-5.457,0.01490,0.0506,0.004320,0.2850,0.894,121.962,176219,Dance/Electronic,2020
4,5,Dua Lipa,Future Nostalgia,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,0.793,0.793,11,-4.521,0.01230,0.0830,0.000000,0.0951,0.679,123.950,183290,Nu-disco,2020


## Combining Cleaned Dataframes
* The 2020 and 2021 dataframes were combined into one dataframe.

In [6]:
df_2020.head(50)

,rank,artist,album,track_name,track_id,energy,danceability,key,loudness,acousticness,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,genre,year
0,1,The Weeknd,After Hours,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,0.730,0.514,1,-5.934,0.00146,0.0598,0.000095,0.0897,0.3340,171.005,200040,R&B/Soul,2020
1,2,Tones And I,Dance Monkey,Dance Monkey,1rgnBhdG2JDFTbYkYRZAku,0.593,0.825,6,-6.401,0.68800,0.0988,0.000161,0.1700,0.5400,98.078,209755,Alternative/Indie,2020
2,3,Roddy Ricch,Please Excuse Me For Being Antisocial,The Box,0nbXyq5TXYPCO7pr3N8S4I,0.586,0.896,10,-6.687,0.10400,0.0559,0.000000,0.7900,0.6420,116.971,196653,Hip-Hop/Rap,2020
3,4,SAINt JHN,Roses (Imanbek Remix),Roses - Imanbek Remix,2Wo6QQD1KMDWeFkkjLqwx5,0.721,0.785,8,-5.457,0.01490,0.0506,0.004320,0.2850,0.8940,121.962,176219,Dance/Electronic,2020
4,5,Dua Lipa,Future Nostalgia,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,0.793,0.793,11,-4.521,0.01230,0.0830,0.000000,0.0951,0.6790,123.950,183290,Nu-disco,2020
5,6,DaBaby,BLAME IT ON BABY,ROCKSTAR (feat. Roddy Ricch),7ytR5pFWmSjzHJIeQkgog4,0.690,0.746,11,-7.956,0.24700,0.1640,0.000000,0.1010,0.4970,89.977,181733,Hip-Hop/Rap,2020
6,7,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.816,0.548,0,-4.209,0.12200,0.0465,0.000000,0.3350,0.5570,95.390,174000,Pop,2020
7,8,Powfu,death bed (coffee for your head),death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,0.431,0.726,8,-8.765,0.73100,0.1350,0.000000,0.6960,0.3480,144.026,173333,Hip-Hop/Rap,2020
8,9,Trevor Daniel,Nicotine,Falling,2rRJrJEo19S2J82BDsQ3F7,0.430,0.784,10,-8.756,0.12300,0.0364,0.000000,0.0887,0.2360,127.087,159382,R&B/Hip-Hop alternative,2020
9,10,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,Someone You Loved,7qEHsqek33rTcFNT9PFqLf,0.405,0.501,1,-5.679,0.75100,0.0319,0.000000,0.1050,0.4460,109.891,182161,Alternative/Indie,2020


In [7]:
# song data columns:
# danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms

df_2021 = df_2021[['year','rank','artist','track_name','danceability', 'energy', 'key', 'loudness', 
         'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()

df_2020 = df_2020[['year','rank','artist','track_name','danceability', 'energy', 'key', 'loudness', 
         'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()

df_combined = pd.concat([df_2021, df_2020])
df_combined.head(100)

,year,rank,artist,track_name,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,2021,1,Olivia Rodrigo,drivers license,0.561,0.431,10,-8.810,0.0578,0.76800,0.000014,0.1060,0.137,143.875,242013
1,2021,2,Lil Nas X,MONTERO (Call Me By Your Name),0.593,0.503,8,-6.725,0.2200,0.29300,0.000000,0.4050,0.710,178.781,137704
2,2021,3,The Kid LAROI,STAY (with Justin Bieber),0.591,0.764,1,-5.484,0.0483,0.03830,0.000000,0.1030,0.478,169.928,141806
3,2021,4,Olivia Rodrigo,good 4 u,0.563,0.664,9,-5.044,0.1540,0.33500,0.000000,0.0849,0.688,166.928,178147
4,2021,5,Dua Lipa,Levitating (feat. DaBaby),0.702,0.825,6,-3.787,0.0601,0.00883,0.000000,0.0674,0.915,102.977,203064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,2020,46,Juice WRLD,Lucid Dreams,0.511,0.566,6,-7.230,0.2000,0.34900,0.000000,0.3400,0.218,83.903,239836
46,2020,47,Ariana Grande,Stuck with U (with Justin Bieber),0.597,0.450,8,-6.658,0.0418,0.22300,0.000000,0.3820,0.537,178.765,228482
47,2020,48,JP Saxe,If the World Was Ending - feat. Julia Michaels,0.464,0.473,1,-10.086,0.1290,0.86600,0.000000,0.1090,0.604,75.801,208687
48,2020,49,Dua Lipa,Physical,0.647,0.844,0,-3.756,0.0457,0.01370,0.000658,0.1020,0.746,146.967,193829


## Creating the artist dataframe
* Combined_df was filtered by unique values in the artist column.
* A new column named "artist_id" was created for a primary key.

In [22]:
artists = pd.DataFrame(df_combined['artist'].unique())
artists.rename(columns={0:'artist'}, inplace=True)
artists.insert(0, 'artist_id', range(1, 1 + len(artists)))
artists.sort_values('artist_id')

,artist_id,artist
0,1,Olivia Rodrigo
1,2,Lil Nas X
2,3,The Kid LAROI
3,4,Dua Lipa
4,5,Justin Bieber
...,...,...
60,61,Regard
61,62,Black Eyed Peas
62,63,THE SCOTTS
63,64,Juice WRLD


## Creating the tracks dataframe
* Columns from combined_df with data pertaining to each track were copied from combined_df into a new dataframe.
* Duplicate track_names were dropped.
* The dataframe was sorted alphabetically by track_name.
* A new column named track_id was added to use as a primary key.

In [9]:
tracks = df_combined[['track_name','artist','danceability', 'energy', 'key', 'loudness', 
         'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
tracks = tracks.drop_duplicates(subset="track_name")
tracks.sort_values('track_name').head(25)

,track_name,artist,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
47,34+35,Ariana Grande,0.830,0.585,0,-6.476,0.0940,0.23700,0.000000,0.2480,0.4850,109.978,173711
21,Adore You,Harry Styles,0.676,0.771,8,-3.675,0.0483,0.02370,0.000007,0.1020,0.5690,99.048,207133
43,Arcade,Duncan Laurence,0.453,0.350,9,-11.639,0.0425,0.82700,0.001220,0.1440,0.2790,72.017,185147
10,Astronaut In The Ocean,Masked Wolf,0.778,0.695,4,-6.865,0.0913,0.17500,0.000000,0.1500,0.4720,149.996,132780
13,Bad Habits,Ed Sheeran,0.807,0.893,11,-3.745,0.0347,0.04510,0.000028,0.3660,0.5370,126.011,230747
32,Bandido,Myke Towers,0.713,0.617,8,-4.637,0.0887,0.12200,0.000000,0.0962,0.6820,168.021,232853
16,Before You Go,Lewis Capaldi,0.459,0.575,3,-4.858,0.0573,0.60400,0.000000,0.0885,0.1830,111.881,215107
9,Beggin',Måneskin,0.714,0.800,11,-4.808,0.0504,0.12700,0.000000,0.3590,0.5890,134.002,211560
7,Blinding Lights,The Weeknd,0.514,0.730,1,-5.934,0.0598,0.00146,0.000095,0.0897,0.3340,171.005,200040
13,Blueberry Faygo,Lil Mosey,0.774,0.554,0,-7.909,0.0383,0.20700,0.000000,0.1320,0.3490,99.034,162547


In [10]:
tracks.insert(0, 'track_id', range(1, 1 + len(tracks)))
tracks.head()

,track_id,track_name,artist,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,1,drivers license,Olivia Rodrigo,0.561,0.431,10,-8.810,0.0578,0.76800,0.000014,0.1060,0.137,143.875,242013
1,2,MONTERO (Call Me By Your Name),Lil Nas X,0.593,0.503,8,-6.725,0.2200,0.29300,0.000000,0.4050,0.710,178.781,137704
2,3,STAY (with Justin Bieber),The Kid LAROI,0.591,0.764,1,-5.484,0.0483,0.03830,0.000000,0.1030,0.478,169.928,141806
3,4,good 4 u,Olivia Rodrigo,0.563,0.664,9,-5.044,0.1540,0.33500,0.000000,0.0849,0.688,166.928,178147
4,5,Levitating (feat. DaBaby),Dua Lipa,0.702,0.825,6,-3.787,0.0601,0.00883,0.000000,0.0674,0.915,102.977,203064


In [11]:
tm = tracks.merge(artists, left_on="artist", right_on="artist")
tm.head()

,track_id,track_name,artist,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_id
0,1,drivers license,Olivia Rodrigo,0.561,0.431,10,-8.810,0.0578,0.768,0.000014,0.1060,0.1370,143.875,242013,1
1,4,good 4 u,Olivia Rodrigo,0.563,0.664,9,-5.044,0.1540,0.335,0.000000,0.0849,0.6880,166.928,178147,1
2,18,deja vu,Olivia Rodrigo,0.442,0.612,2,-7.222,0.1120,0.584,0.000006,0.3700,0.1780,180.917,215507,1
3,29,traitor,Olivia Rodrigo,0.380,0.339,3,-7.885,0.0338,0.691,0.000000,0.1200,0.0849,100.607,229227,1
4,2,MONTERO (Call Me By Your Name),Lil Nas X,0.593,0.503,8,-6.725,0.2200,0.293,0.000000,0.4050,0.7100,178.781,137704,2


In [12]:
tracks = tm[['track_id','artist_id','track_name','danceability', 'energy', 'key', 'loudness', 
         'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
tracks.head()

,track_id,artist_id,track_name,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,1,1,drivers license,0.561,0.431,10,-8.810,0.0578,0.768,0.000014,0.1060,0.1370,143.875,242013
1,4,1,good 4 u,0.563,0.664,9,-5.044,0.1540,0.335,0.000000,0.0849,0.6880,166.928,178147
2,18,1,deja vu,0.442,0.612,2,-7.222,0.1120,0.584,0.000006,0.3700,0.1780,180.917,215507
3,29,1,traitor,0.380,0.339,3,-7.885,0.0338,0.691,0.000000,0.1200,0.0849,100.607,229227
4,2,2,MONTERO (Call Me By Your Name),0.593,0.503,8,-6.725,0.2200,0.293,0.000000,0.4050,0.7100,178.781,137704


## Creating the chart dataframe
* The "year", "rank", "artist", and "track_name" columns from combined_df were copied into a new dataframe.


In [13]:
chart = df_combined[['year','rank','artist','track_name']]
chart.head()

,year,rank,artist,track_name
0,2021,1,Olivia Rodrigo,drivers license
1,2021,2,Lil Nas X,MONTERO (Call Me By Your Name)
2,2021,3,The Kid LAROI,STAY (with Justin Bieber)
3,2021,4,Olivia Rodrigo,good 4 u
4,2021,5,Dua Lipa,Levitating (feat. DaBaby)


In [14]:
cm=chart.merge(tracks, on='track_name')
cm = cm.merge(artists, left_on='artist',right_on='artist')
cm.head()

,year,rank,artist,track_name,track_id,artist_id_x,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_id_y
0,2021,1,Olivia Rodrigo,drivers license,1,1,0.561,0.431,10,-8.810,0.0578,0.768,0.000014,0.1060,0.1370,143.875,242013,1
1,2021,4,Olivia Rodrigo,good 4 u,4,1,0.563,0.664,9,-5.044,0.1540,0.335,0.000000,0.0849,0.6880,166.928,178147,1
2,2021,18,Olivia Rodrigo,deja vu,18,1,0.442,0.612,2,-7.222,0.1120,0.584,0.000006,0.3700,0.1780,180.917,215507,1
3,2021,29,Olivia Rodrigo,traitor,29,1,0.380,0.339,3,-7.885,0.0338,0.691,0.000000,0.1200,0.0849,100.607,229227,1
4,2021,2,Lil Nas X,MONTERO (Call Me By Your Name),2,2,0.593,0.503,8,-6.725,0.2200,0.293,0.000000,0.4050,0.7100,178.781,137704,2


In [15]:
chart = cm[['year','rank','track_id','artist_id_y']].copy()
chart.rename(columns={'artist_id_y':'artist_id'}, inplace=True)
chart.head()

,year,rank,track_id,artist_id
0,2021,1,1,1
1,2021,4,4,1
2,2021,18,18,1
3,2021,29,29,1
4,2021,2,2,2


In [16]:
chart.sort_values(['year','rank'])

,year,rank,track_id,artist_id
22,2020,1,8,7
65,2020,2,51,36
66,2020,3,52,37
67,2020,4,53,38
10,2020,5,41,4
...,...,...,...,...
61,2021,46,46,32
62,2021,47,47,33
49,2021,48,48,23
63,2021,49,49,34


# Load
describe how we put the data into the database

In [17]:
# Connection to database
# Change username, password, host to match your config. Password should be imported from config.py
protocol = 'postgresql'
username = 'postgres'
password = db_password
host = 'localhost'
port = 5432
database_name = 'spotify_top50'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [18]:
inspector = inspect(engine)
inspector.get_table_names()

['chart', 'track', 'artist']

In [23]:
# Load dataframes into database
artists.to_sql(name='artist', con=engine, if_exists='append', index=False)
tracks.to_sql(name='track', con=engine, if_exists='append', index=False)
chart.to_sql(name='chart', con=engine, if_exists='append', index=False)

100

In [27]:
# Example of how to work with the data using pandas
pd.read_sql_query('select * from chart', con=engine).head() 

,year,rank,artist_id,track_id
0,2021,1,1,1
1,2021,4,1,4
2,2021,18,1,18
3,2021,29,1,29
4,2021,2,2,2
